'''
This a test documentation for creating and loading tables to sales_database
'''

In [12]:
import pandas as pd
import tabula
import numpy as np
import requests
import json
import dateparser
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from dateutil.parser import parse

# table 'dim_card_details'
db_engine2 = DatabaseConnector().init_db_engine()
extractor2 = DataExtractor(db_engine2)
new_table = extractor2.retrieve_pdf_data("https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf")
cleaner = DataCleaning(new_table)
cards = cleaner.clean_card_data()
# cards.head(10)
# cards.info()
new_engine = DatabaseConnector()
new_engine.upload_to_db(cards, 'dim_card_details')

In [11]:
import pandas as pd
import tabula
import numpy as np
import requests
import json
import dateparser
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from dateutil.parser import parse

# table 'orders_table'
db_engine = DatabaseConnector()
conn = DatabaseConnector().init_db_engine()
ext = DataExtractor(conn)
ord = ext.read_rds_table('orders_table')
a = DataCleaning(ord)
ord_table = a.clean_orders_data()
ord_table.head(2)

new_engine = DatabaseConnector()
new_engine.upload_to_db(ord_table, 'orders_table')

In [4]:
import pandas as pd
import numpy as np
import requests
import json
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

# df = pd.read_json("https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json")
# df.head(1)
# # df.info()

response = requests.get("https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json")
data = json.loads(response.text)
df = pd.DataFrame(data)
df.head(5)


,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [ ]:
data = {
    "date_payment_confirmed" : ["December 2000 01","December 2021 17","May 1998 09","UZGSD0AEBT","OE3KONN2V6","2015-11-25","1995-08-24"]
}

df = pd.DataFrame(data)
def parse_date(date_str):
    try:
        parsed_date = dateparser.parse(date_str)
        return parsed_date
    except Exception as e:
        # print(f"Error parsing date '{date_str}': {e}")
        return pd.NaT
            
df['date_payment_confirmed'] = df['date_payment_confirmed'].apply(parse_date)
df['date_payment_confirmed'] = pd.to_datetime(df['date_payment_confirmed'], format="ISO8601", errors='coerce')
df.head(7)